In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
# relative imports
from uncertainty_aware_diagnosis import ICD10data

# absolute imports
from torch.utils.data import DataLoader

# paths
train_csv = "./data/train.csv"
val_csv = "./data/val.csv"
test_csv = "./data/test.csv"

In [6]:
# variables
batch_size = 32

In [ ]:
ds_train = ICD10data(train_csv)
ds_val = ICD10data(val_csv)
ds_test = ICD10data(test_csv)

train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(ds_val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(ds_test, batch_size=batch_size, shuffle=True)